In [ ]:
!pip install torch transformers datasets nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the model and tokenizer
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


def preprocess(data):
    inputs = tokenizer(data['question'], truncation=True, padding='max_length', max_length=128)
    labels = tokenizer(data['query'], truncation=True, padding='max_length', max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs


# Example dataset (load Spider/WikiSQL)
from datasets import load_dataset
dataset = load_dataset("spider")


print(dataset['train'].column_names)


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks']


In [ ]:
print(dataset["train"])

Dataset({
    features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
    num_rows: 7000
})


In [ ]:
# Tokenize dataset
tokenized_dataset = dataset.map(preprocess, batched=True)

# Fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
)
trainer.train()

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.377856
2,1.156200,0.328890
3,0.349100,0.313717
4,0.299100,0.306492
5,0.279900,0.304736


TrainOutput(global_step=2190, training_loss=0.4997856140136719, metrics={'train_runtime': 2298.8326, 'train_samples_per_second': 15.225, 'train_steps_per_second': 0.953, 'total_flos': 5328381542400000.0, 'train_loss': 0.4997856140136719, 'epoch': 5.0})

In [ ]:
# Save the model and tokenizer
model.save_pretrained("drive/MyDrive/AI_lab/saved_model_base")
tokenizer.save_pretrained("drive/MyDrive/AI_lab/saved_model_base")


('drive/MyDrive/AI_lab/saved_model_base/tokenizer_config.json',
 'drive/MyDrive/AI_lab/saved_model_base/special_tokens_map.json',
 'drive/MyDrive/AI_lab/saved_model_base/spiece.model',
 'drive/MyDrive/AI_lab/saved_model_base/added_tokens.json')

In [ ]:
def generate_sql(query):
    # Tokenize the input query
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding="max_length", max_length=128)

    # Generate SQL
    outputs = model.generate(inputs.input_ids, max_length=50, num_beams=4, early_stopping=True)

    # Decode the output
    sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return sql_query

# Example query
natural_language_query = "Extract all product names from product table whose prices are greater than 500"
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)


Generated SQL
 SELECT product_name FROM products WHERE product_price  >  500


In [ ]:
import pandas as pd
import random

# Define dimensions, weight, and price mapping for products
product_info = {
    "Laptop": {"dimensions": "35x25x2", "weight": 1.5, "price": 1000},
    "Smartphone": {"dimensions": "15x7x1", "weight": 0.2, "price": 500},
    "Tablet": {"dimensions": "25x18x1", "weight": 0.4, "price": 300},
    "Headphones": {"dimensions": "20x15x5", "weight": 0.3, "price": 100},
    "Monitor": {"dimensions": "60x40x10", "weight": 5.0, "price": 200},
    "Keyboard": {"dimensions": "45x15x3", "weight": 0.8, "price": 50},
    "Mouse": {"dimensions": "10x5x3", "weight": 0.1, "price": 20},
    "Smartwatch": {"dimensions": "5x5x1", "weight": 0.2, "price": 150},
    "Camera": {"dimensions": "15x10x8", "weight": 0.7, "price": 400},
    "Printer": {"dimensions": "45x30x20", "weight": 5.0, "price": 300},
    "Desk Lamp": {"dimensions": "30x15x15", "weight": 1.0, "price": 75},
    "Router": {"dimensions": "20x15x5", "weight": 0.6, "price": 80},
    "Power Bank": {"dimensions": "10x6x2", "weight": 0.3, "price": 40},
    "External Hard Drive": {"dimensions": "12x8x2", "weight": 0.5, "price": 100},
    "Flash Drive": {"dimensions": "5x1x1", "weight": 0.02, "price": 15},
    "Speaker": {"dimensions": "20x15x10", "weight": 1.5, "price": 120},
    "Webcam": {"dimensions": "7x5x2", "weight": 0.1, "price": 60},
    "Gaming Console": {"dimensions": "30x25x10", "weight": 2.5, "price": 300},
    "Air Conditioner": {"dimensions": "75x50x30", "weight": 30.0, "price": 800},
    "Refrigerator": {"dimensions": "180x80x70", "weight": 80.0, "price": 1200},
    "Washing Machine": {"dimensions": "60x60x90", "weight": 60.0, "price": 700},
    "Microwave": {"dimensions": "50x35x40", "weight": 12.0, "price": 150},
    "Electric Kettle": {"dimensions": "20x20x25", "weight": 1.2, "price": 70},
    "Coffee Maker": {"dimensions": "20x15x30", "weight": 1.5, "price": 90},
    "Juicer": {"dimensions": "20x15x25", "weight": 1.0, "price": 50},
    "Mixer Grinder": {"dimensions": "25x20x25", "weight": 2.0, "price": 120},
    "Rice Cooker": {"dimensions": "25x25x25", "weight": 3.0, "price": 100},
    "Hair Dryer": {"dimensions": "30x20x10", "weight": 0.5, "price": 40},
    "Straightener": {"dimensions": "25x7x5", "weight": 0.4, "price": 30},
    "Electric Toothbrush": {"dimensions": "5x2x2", "weight": 0.1, "price": 25},
    "Blender": {"dimensions": "30x20x15", "weight": 2.5, "price": 150},
    "Smart Bulb": {"dimensions": "6x6x12", "weight": 0.2, "price": 20},
    "Electric Fan": {"dimensions": "50x50x20", "weight": 2.5, "price": 100},
    "Vacuum Cleaner": {"dimensions": "40x30x25", "weight": 5.0, "price": 250},
    "Water Purifier": {"dimensions": "40x30x20", "weight": 7.5, "price": 300},
    "Dishwasher": {"dimensions": "60x60x85", "weight": 40.0, "price": 600},
    "Iron": {"dimensions": "30x12x15", "weight": 1.0, "price": 50},
    "Induction Cooktop": {"dimensions": "30x25x5", "weight": 2.0, "price": 120},
    "Electric Cooker": {"dimensions": "25x25x25", "weight": 3.0, "price": 150},
    "Food Processor": {"dimensions": "30x25x15", "weight": 3.0, "price": 200}
}

# Read the existing order data from the Excel file
order_data = pd.read_excel("updated_orders_data.xlsx")

# Keep only the necessary columns (Order ID and Product Name)
order_data = order_data[["Order ID", "Product Name"]]

# Add dimensions, weight, and price to the order data
def add_dimensions_weight_price(row):
    product_name = row["Product Name"]
    if product_name in product_info:
        dimensions = product_info[product_name]["dimensions"]
        weight = product_info[product_name]["weight"]
        price = product_info[product_name]["price"]
    else:
        # Default values if not found in the map
        dimensions = "Unknown"
        weight = random.uniform(1, 10)  # Random weight for unknown products
        price = random.uniform(50, 500)  # Random price for unknown products
    row["Dimensions (cm)"] = dimensions
    row["Weight (kg)"] = weight
    row["Price ($)"] = price
    return row

# Apply the function to add dimensions, weight, and price to each row
order_data = order_data.apply(add_dimensions_weight_price, axis=1)

# Save the updated order data to a new Excel file
output_file = "updated_order_data_with_dimensions_weight_price.xlsx"
order_data.to_excel(output_file, index=False)

print(f"Updated order data with dimensions, weight, and price saved to {output_file}.")


In [ ]:
natural_language_query = "Extract all the unique product names from Order_Data"
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)

Generated SQL
 SELECT DISTINCT product_name FROM Order_Data


In [ ]:
natural_language_query = str(input("enter:"))
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)

enter:Package_data has a primary key order_id for every product with weight>5 in prduct_data extract the order_id
Generated SQL
 SELECT order_id FROM products WHERE weight  >  5 IN prduct_data


In [ ]:
natural_language_query = str(input("enter:"))
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)

enter:Give me all the companies with rating>4 from company _data
Generated SQL
 SELECT DISTINCT company FROM company WHERE rating  >  4


In [ ]:
natural_language_query = str(input("enter:"))
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)

enter:Give me a list of all customers who live in munich from customer data
Generated SQL
 SELECT customer_details FROM customers WHERE city_code  =  'munich'


In [ ]:
natural_language_query = str(input("enter:"))
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)

enter:Extract all Order_ID from Order_Data where the Quantity is greater than 10 and less than 15
Generated SQL
 SELECT Order_ID FROM Order_Data WHERE Quantity  >  10 INTERSECT SELECT Order_id FROM Orders WHERE Quantity <  15


In [ ]:
natural_language_query = str(input("enter:"))
sql_query = generate_sql(natural_language_query)
print("Generated SQL\n", sql_query)